In [8]:
#Permanent cell 1
import arcpy
import pandas as pd
import sqlite3
import math
import numpy as np
import os

In [9]:
#Permanent cell 2
def sql_to_df(sql,model):
    con = sqlite3.connect(model)
    df = pd.read_sql(sql, con)
    con.close()
    return df

def execute_sql(sqls,model):
    con = sqlite3.connect(model)
    cur = con.cursor()
    if type(sqls) == list:
        for sql in sqls:
            cur.execute(sql)
    else:         
        cur.execute(sqls)
    cur.close()
    con.commit()
    con.close()

In [10]:
#Permanent cell 3
# User Input, to move to separate sheet so no permanent cell
#stop trace if more pipes than max_steps traced from catchment, must be an endless loop. 
max_steps = 1000 

update_field_in_model = True
update_field = 'Description'

output_folder = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\Rawn_Tool\Output"
model_path = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\NSSA_Base_2018pop.sqlite"
sewer_area = 'NSSA'
pop_book = r"\\prdsynfile01\LWS_Modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\04_DATA\01. POPULATION\MPF4_Temp_Hold\NSSA_Master_Population_File_4_No_2237_ResArea.xlsx"
pop_sheet = 'MPF Update 4'
model = 'NSSA'
gdb_name = 'RAWN.gdb'


In [11]:
#Permanent cell 4
#Set up column names

years = [2060,2070,2080,2090,2100]
categories = ['res','com','ind','inst','infl','infi']

mpf_col_dict = {}

area_col_dict = {}
area_col_dict['res'] = 'Area_Res'
area_col_dict['com'] = 'Area_Com'
area_col_dict['ind'] = 'Area_Ind'
area_col_dict['inst'] = 'Area_Inst'
area_col_dict['ini'] = 'Area_Total'

per_unit_dict = {}
per_unit_dict['res'] = 320
per_unit_dict['com'] = 33700 
per_unit_dict['ind'] = 56200
per_unit_dict['inst'] = 33700
per_unit_dict['infl'] = 5600
per_unit_dict['infi'] = 5600

header_dict = {}
# header_dict['gen'] = ['GENERAL INFO',['TYPE','MODELID','CATCHMENT','ID','YEAR','LOCATION']]
header_dict['gen'] = ['GENERAL INFO',['TYPE','CATCHMENT','YEAR','LOCATION']]
header_dict['res'] = ['RESIDENTIAL',['AREA (Ha)','POPULATION','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
header_dict['com'] = ['COMMERCIAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
header_dict['ind'] = ['INDUSTRIAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
header_dict['inst'] = ['INSTITUTIONAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
header_dict['ini'] = ['INFLOW / INFILTRATION',['AREA (Ha)','INFLOW (L/s)','INFILTRATION (L/s)']]
header_dict['flow'] = ['FLOWS',['AVG. SAN. FLOW (L/s)','ADWF (L/s)','PWWF (L/s)']]

avg_calc_dict = {}
avg_calc_dict['res'] = ['RESIDENTIAL','POPULATION','AVG. FLOW (L/s)']
avg_calc_dict['com'] = ['COMMERCIAL','AREA (Ha)','AVG. FLOW (L/s)']
avg_calc_dict['ind'] = ['INDUSTRIAL','AREA (Ha)','AVG. FLOW (L/s)']
avg_calc_dict['inst'] = ['INSTITUTIONAL','AREA (Ha)','AVG. FLOW (L/s)']
avg_calc_dict['infl'] = ['INFLOW / INFILTRATION','AREA (Ha)','INFLOW (L/s)']
avg_calc_dict['infi'] = ['INFLOW / INFILTRATION','AREA (Ha)','INFILTRATION (L/s)']

header_tuples = []
for header in header_dict:
    for sub_header in (header_dict[header][1]):
        header_tuples.append((header_dict[header][0],sub_header))
header_tuples

# columns_multiindex = pd.MultiIndex.from_tuples(header_tuples,names=['Category', 'Subcategory'])
columns_multiindex = pd.MultiIndex.from_tuples(header_tuples)
df_template = pd.DataFrame(columns=columns_multiindex)





In [12]:
#Permanent cell 5
#Import population
pop_df = pd.read_excel(pop_book,sheet_name=pop_sheet,dtype={'Catchment': str})#[['Catchment','Year','Pop_Total']]
pop_df.rename(columns={"Pop_Total": "Population"},inplace=True)
pop_df = pop_df[['Catchment','Year','Pop_ResLD','Pop_ResHD','Pop_Mixed','Population','Area_ResLD','Area_ResHD','Area_Mixed','Area_Com','Area_Ind','Area_Inst']]
pop_df['Area_Res'] = pop_df.Area_ResLD + pop_df.Area_ResHD + pop_df.Area_Mixed
pop_df['Area_Total'] = pop_df.Area_ResLD + pop_df.Area_ResHD + pop_df.Area_Mixed + pop_df.Area_Com + pop_df.Area_Ind + pop_df.Area_Inst
pop_df['Population_Sum_Check'] = pop_df.Pop_ResLD + pop_df.Pop_ResHD + pop_df.Pop_Mixed
pop_sum_total_col = int(pop_df.Population.sum())
pop_sum_sub_cols = int(pop_df.Pop_ResLD.sum() + pop_df.Pop_ResHD.sum() + pop_df.Pop_Mixed.sum())
pop_df['Key'] = sewer_area + '@' + pop_df.Catchment + '@' + pop_df['Year'].astype(str)
pop_df.set_index('Key',inplace=True)

if pop_sum_total_col != pop_sum_sub_cols:
      raise ValueError("Error. The sum of 'Population' (" + str(pop_sum_total_col) + ") is different than the sum of 'Pop_ResLD' + 'Pop_ResHD' + 'Pop_Mixed' (" + str(pop_sum_sub_cols) + ")") 


In [13]:
#Permanent cell 6
#Import model data

node_types = {}
node_types[1] = 'Manhole'
node_types[2] = 'Basin'
node_types[3] = 'Outlet'
node_types[4] = 'Junction'
node_types[5] = 'Soakaway'
node_types[6] = 'River Junction'

sql = "SELECT catchid AS Catchment, nodeid AS Connected_Node FROM msm_Catchcon WHERE Active = 1"
catchments = sql_to_df(sql,model_path)

sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], uplevel AS Outlet_Level FROM msm_Link WHERE Active = 1"
lines = sql_to_df(sql,model_path)

sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], invertlevel AS Outlet_Level FROM msm_Orifice WHERE Active = 1"
orifices = sql_to_df(sql,model_path)
lines = pd.concat([lines,orifices])

sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], invertlevel AS Outlet_Level FROM msm_Valve WHERE Active = 1"
valves = sql_to_df(sql,model_path)
lines = pd.concat([lines,valves])

sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], crestlevel AS Outlet_Level FROM msm_Weir WHERE Active = 1"
weirs = sql_to_df(sql,model_path)
lines = pd.concat([lines,weirs])

sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], startlevel AS Outlet_Level FROM msm_Pump WHERE Active = 1"
pumps = sql_to_df(sql,model_path)
lines = pd.concat([lines,pumps])

lines['Outlet_Level'].fillna(-9999, inplace=True)

sql = "SELECT muid, acronym, assetname FROM msm_Node WHERE active = 1"
node_id_df = sql_to_df(sql,model_path)
node_id_df = node_id_df[(node_id_df.assetname.str[:2]=='MH') & (node_id_df.assetname.str.len() > 2) & (node_id_df.acronym.notna())]
node_id_df.rename(columns={'muid':'Node'},inplace=True)
node_id_df['ID'] = node_id_df.acronym + '_' + node_id_df.assetname
node_id_df = node_id_df[['Node','ID']]

In [15]:
#Permanent cell 7
#Trace the model

accumulated_catchment_set = set()

for index1, row1 in catchments.iterrows():
    catchment = row1['Catchment']
    nodes = [row1['Connected_Node']]
    start_node = row1['Connected_Node']
    steps = 0
    
    accumulated_catchment_set.add((start_node,catchment))
        
    while steps <= max_steps:
        steps += 1
        downstream_df = lines[lines['From'].isin(nodes)]

        if len(downstream_df) > 0:
            nodes = list(downstream_df.To.unique())

            nodes = [node for node in nodes if len(node)>0]
            for node in nodes:
                accumulated_catchment_set.add((node,catchment))       
        else:
            break
        if steps == max_steps:
            raise ValueError("Maximum steps were reached, indicating a loop. Start catchment is '" + catchment + "'")
           
        accumulated_catchment_set.add((node,catchment))
        
accumulation_df = pd.DataFrame(accumulated_catchment_set,columns=['Node','Catchment'])
accumulation_df = pd.merge(accumulation_df,node_id_df,how='inner',on=['Node'])
data = {
    ('GENERAL INFO', 'CATCHMENT'): accumulation_df.Catchment,
    ('GENERAL INFO', 'NODE'): accumulation_df.Node,
    ('GENERAL INFO', 'ID'): accumulation_df.ID,
}

# Create a DataFrame with MultiIndex columns
accumulation_df = pd.DataFrame(data)



In [18]:
#Permanent cell 8
#Calculate RAWN

catchments = list(pop_df.Catchment.unique())

catchment_df = df_template.copy()
for catchment in catchments:
    for year in years:
        key = model + '@' + catchment + '@' + str(year)
        catchment_df.loc[key,('GENERAL INFO','TYPE')] = 'UNKNOWN'
        catchment_df.loc[key,('GENERAL INFO','CATCHMENT')] = catchment
        catchment_df.loc[key,('GENERAL INFO','YEAR')] = year
        catchment_df.loc[key,('GENERAL INFO','LOCATION')] = model
        for area_col_dict_key in area_col_dict:
            catchment_df.loc[key,(header_dict[area_col_dict_key][0],'AREA (Ha)')] = pop_df.loc[key,area_col_dict[area_col_dict_key]]
        catchment_df.loc[key,('RESIDENTIAL','POPULATION')] = pop_df.loc[key,'Population']
        san_flow = 0
        adwf = 0
        for avg_calc_dict_key in avg_calc_dict:
            input1 = catchment_df.loc[key,(avg_calc_dict[avg_calc_dict_key][0],avg_calc_dict[avg_calc_dict_key][1])]
            input2 = per_unit_dict[avg_calc_dict_key]
            avg_flow = input1 * input2 / 86400
            adwf += avg_flow
            if avg_calc_dict_key not in ['infl','infi']:
                san_flow += avg_flow
            catchment_df.loc[key,(avg_calc_dict[avg_calc_dict_key][0],avg_calc_dict[avg_calc_dict_key][2])] = avg_flow
        catchment_df.loc[key,('FLOWS','AVG. SAN. FLOW (L/s)')] = san_flow
        catchment_df.loc[key,('FLOWS','ADWF (L/s)')] = adwf

        
catchment_node_df = accumulation_df.merge(catchment_df,on=[('GENERAL INFO','CATCHMENT')],how='inner')
node_df = catchment_node_df.copy()
node_df.drop(columns=[('GENERAL INFO','CATCHMENT')],inplace=True)
node_df = node_df.groupby([('GENERAL INFO','NODE'),('GENERAL INFO','TYPE'),('GENERAL INFO','YEAR'),('GENERAL INFO','LOCATION'),('GENERAL INFO','ID')]).sum()
node_df.reset_index(inplace=True)
node_df[('RESIDENTIAL','PEAK FLOW (L/s)')] = (1 + 14 / (4 + (node_df[('RESIDENTIAL','POPULATION')] / 1000) ** 0.5)) * node_df[('RESIDENTIAL','AVG. FLOW (L/s)')]
node_df[('COMMERCIAL','PEAK FLOW (L/s)')] = (1 + 14 / (4 + (per_unit_dict['com'] * node_df[('COMMERCIAL','AREA (Ha)')]/(per_unit_dict['res'] * 1000)) ** 0.5))*node_df[('COMMERCIAL','AVG. FLOW (L/s)')]*0.8
node_df[('INSTITUTIONAL','PEAK FLOW (L/s)')] = (1 + 14 / (4 + (per_unit_dict['inst'] * node_df[('INSTITUTIONAL','AREA (Ha)')] / (per_unit_dict['res'] * 1000)) ** 0.5)) * node_df[('INSTITUTIONAL','AVG. FLOW (L/s)')]

mask = node_df[('INDUSTRIAL', 'AREA (Ha)')] != 0 #Avoid error from log(0)
node_df.loc[mask, ('INDUSTRIAL', 'PEAK FLOW (L/s)')] = (
    0.8 * (1 + 14 / (4 + (node_df[('INDUSTRIAL', 'AREA (Ha)')][mask] * per_unit_dict['ind'] / (per_unit_dict['res'] * 1000)) ** 0.5)) *
    np.where(
        node_df[('INDUSTRIAL', 'AREA (Ha)')][mask] < 121,
        1.7,
        2.505 - 0.1673 * np.log(node_df[('INDUSTRIAL', 'AREA (Ha)')][mask])
    ) * node_df[('INDUSTRIAL', 'AVG. FLOW (L/s)')][mask]
)

node_df[('FLOWS','PWWF (L/s)')] = (
    node_df[('RESIDENTIAL','PEAK FLOW (L/s)')] +
    node_df[('COMMERCIAL','PEAK FLOW (L/s)')] +
    node_df[('INDUSTRIAL','PEAK FLOW (L/s)')] +
    node_df[('INSTITUTIONAL','PEAK FLOW (L/s)')] +
    node_df[('INFLOW / INFILTRATION','INFLOW (L/s)')] +
    node_df[('INFLOW / INFILTRATION','INFILTRATION (L/s)')]
)

excel_folder = output_folder + '\\Excel'
if not os.path.isdir(excel_folder): os.makedirs(excel_folder) 
for id in node_df[('GENERAL INFO','ID')].unique():    
    node_single_df = node_df[node_df[('GENERAL INFO','ID')]==id]
    id = id.replace('/','-') if '/' in id else id
    node_single_df.to_excel(excel_folder + '\\' + id + '.xlsx')


In [17]:
node_single_df

GENERAL INFO                 ...                FLOWS                        
             NODE     TYPE  YEAR  ... AVG. SAN. FLOW (L/s)  ADWF (L/s)  PWWF (L/s)
2340         9984  UNKNOWN  2060  ...           218.632300  257.272198  541.889261
2341         9984  UNKNOWN  2070  ...           241.652049  280.291948  583.556508
2342         9984  UNKNOWN  2080  ...           264.671799  303.311698  624.453387
2343         9984  UNKNOWN  2090  ...           287.691548  326.331447  664.671275
2344         9984  UNKNOWN  2100  ...           310.711298  349.351197  704.284485

[5 rows x 24 columns]

In [37]:
dir(sym)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__from_scripting_arc_object__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_arc_object', '_createRendererType', '_create_classify_colorizer', '_create_graduated_colors', '_create_graduated_symbols', '_create_single_symbol', '_create_stretch_colorizer', '_create_unclassed_colors', '_create_unique_value', '_create_unique_value_colorizer', '_getColorizer', '_getRenderer', '_renderer', '_rendererType', '_updateColorizer', '_updateRenderer', '_visualizer', 'renderer', 'updateRenderer']

In [39]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps('Map')[0]
for lyr in m.listLayers():
  if lyr.isFeatureLayer:
    print (lyr.name)
    sym = lyr.symbology
    if lyr.name == 'msm_CatchCon':
        sym.renderer.symbol.color = {'RGB': [255, 0, 0]}  # Red color
        lyr.symbology = sym
aprx.save()
        

msm_CatchCon
msm_Valve
msm_Orifice
msm_Weir
msm_Pump
msm_Link
msm_CatchCon
msm_Catchment_Dissolve_Single
msm_Catchment


In [23]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
out_path = r'J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\Rawn_Tool\Output' + '\\' + gdb_name
arcpy.env.workspace = out_path
sr = arcpy.SpatialReference(26910)
layers = ['msm_CatchCon']
arcpy.env.overwriteOutput = True
for layer in layers:
    arcpy.conversion.FeatureClassToFeatureClass(model_path + '\\' + layer, out_path, layer)
    arcpy.DefineProjection_management(layer, sr)


In [130]:
#Permanent cell 9
#Import GIS from the model

arcpy.management.CreateFileGDB(output_folder, gdb_name)
out_path = r'J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\Rawn_Tool\Output' + '\\' + gdb_name
arcpy.env.workspace = out_path
sr = arcpy.SpatialReference(26910)

layers = ['msm_CatchCon','msm_Catchment','msm_Link','msm_Node','msm_Pump','msm_Weir','msm_Orifice','msm_Valve']

for layer in layers:
#     arcpy.conversion.FeatureClassToFeatureClass(model_path + '\\' + layer, out_path, layer)
   
    arcpy.management.MakeFeatureLayer(model_path + '\\' + layer, "temp_layer", "Active = 1")  
    arcpy.conversion.FeatureClassToFeatureClass("temp_layer", out_path, layer)
    arcpy.management.Delete("temp_layer")
    arcpy.DefineProjection_management(layer, sr)
    

arcpy.management.AddField('msm_catchment', "Drains_To", "TEXT")



<Result 'msm_Catchment'>

In [131]:
#Permanent cell 10
#Dissolve catchments

nodes = list(accumulation_df[('GENERAL INFO','NODE')].unique())
for i, node in enumerate(nodes):
    print('Dissolving for node ' + str(i) + ' of ' + str(len(nodes)) + ' at time ' + str(datetime.datetime.now()))
    catchment_df = accumulation_df[accumulation_df[('GENERAL INFO','NODE')]==node]
    catchments = list(catchment_df[('GENERAL INFO','CATCHMENT')].unique())
    arcpy.management.CalculateField('msm_Catchment', "Drains_To", "''", "PYTHON3")
    with arcpy.da.UpdateCursor('msm_catchment', ['muid', 'Drains_To']) as cursor:
        for row in cursor:
            if row[0] in catchments:
                row[1] = node
                cursor.updateRow(row)

    query = "Drains_To = 'Test'"
    arcpy.management.MakeFeatureLayer('msm_catchment', "temp_layer", "Drains_To = '" + node + "'")
    dissolve_output = out_path + '\\msm_Catchment_Dissolve_Single'
    arcpy.management.Dissolve("temp_layer", dissolve_output, "Drains_To", "", "MULTI_PART")
    arcpy.management.Delete("temp_layer")

    arcpy.conversion.FeatureClassToFeatureClass('msm_Catchment_Dissolve_Single', out_path, 'Node_Catchment_' + node)



Dissolving for node 0 of 466 at time 2024-05-10 08:24:55.665100
Dissolving for node 1 of 466 at time 2024-05-10 08:25:01.888875
Dissolving for node 2 of 466 at time 2024-05-10 08:25:08.672169
Dissolving for node 3 of 466 at time 2024-05-10 08:25:15.605602
Dissolving for node 4 of 466 at time 2024-05-10 08:25:22.106634
Dissolving for node 5 of 466 at time 2024-05-10 08:25:29.173191
Dissolving for node 6 of 466 at time 2024-05-10 08:25:36.103622
Dissolving for node 7 of 466 at time 2024-05-10 08:25:43.058075
Dissolving for node 8 of 466 at time 2024-05-10 08:25:50.473956
Dissolving for node 9 of 466 at time 2024-05-10 08:25:57.442421
Dissolving for node 10 of 466 at time 2024-05-10 08:26:04.805252
Dissolving for node 11 of 466 at time 2024-05-10 08:26:12.039965
Dissolving for node 12 of 466 at time 2024-05-10 08:26:19.694068
Dissolving for node 13 of 466 at time 2024-05-10 08:26:27.034878
Dissolving for node 14 of 466 at time 2024-05-10 08:26:34.990260
Dissolving for node 15 of 466 at ti

Dissolving for node 249 of 466 at time 2024-05-10 09:01:28.501491
Dissolving for node 250 of 466 at time 2024-05-10 09:01:39.146326
Dissolving for node 251 of 466 at time 2024-05-10 09:01:50.301633
Dissolving for node 252 of 466 at time 2024-05-10 09:02:01.163669
Dissolving for node 253 of 466 at time 2024-05-10 09:02:10.966726
Dissolving for node 254 of 466 at time 2024-05-10 09:02:21.420385
Dissolving for node 255 of 466 at time 2024-05-10 09:02:31.656840
Dissolving for node 256 of 466 at time 2024-05-10 09:02:41.637056
Dissolving for node 257 of 466 at time 2024-05-10 09:02:52.732302
Dissolving for node 258 of 466 at time 2024-05-10 09:03:03.426177
Dissolving for node 259 of 466 at time 2024-05-10 09:03:14.828707
Dissolving for node 260 of 466 at time 2024-05-10 09:03:25.611969
Dissolving for node 261 of 466 at time 2024-05-10 09:03:37.201672
Dissolving for node 262 of 466 at time 2024-05-10 09:03:47.826483
Dissolving for node 263 of 466 at time 2024-05-10 09:03:58.567402
Dissolving

In [138]:
#Permanent cell 11
#Append individual dissolved catchments to one layer.

nodes = list(accumulation_df[('GENERAL INFO','NODE')].unique())
for i, node in enumerate(nodes[408:]):    
    print('Appending for node ' + str(i) + ' of ' + str(len(nodes)) + ' at time ' + str(datetime.datetime.now()))
    if i == 0:
        arcpy.conversion.FeatureClassToFeatureClass('Node_Catchment_' + node, out_path, 'Node_Catchment')
    else:
        arcpy.management.Append('Node_Catchment_' + node, "Node_Catchment", "NO_TEST")
    arcpy.management.Delete('Node_Catchment_' + node)

Appending for node 0 of 466 at time 2024-05-10 14:37:37.026020
Appending for node 1 of 466 at time 2024-05-10 14:37:44.709064
Appending for node 2 of 466 at time 2024-05-10 14:37:47.337474
Appending for node 3 of 466 at time 2024-05-10 14:37:49.882808
Appending for node 4 of 466 at time 2024-05-10 14:37:52.131870
Appending for node 5 of 466 at time 2024-05-10 14:37:54.349903
Appending for node 6 of 466 at time 2024-05-10 14:37:57.453749
Appending for node 7 of 466 at time 2024-05-10 14:37:59.886980
Appending for node 8 of 466 at time 2024-05-10 14:38:02.080992
Appending for node 9 of 466 at time 2024-05-10 14:38:04.573277
Appending for node 10 of 466 at time 2024-05-10 14:38:06.889400
Appending for node 11 of 466 at time 2024-05-10 14:38:09.293605
Appending for node 12 of 466 at time 2024-05-10 14:38:12.188259
Appending for node 13 of 466 at time 2024-05-10 14:38:14.962803
Appending for node 14 of 466 at time 2024-05-10 14:38:17.457089
Appending for node 15 of 466 at time 2024-05-10 14

In [137]:
nodes[408]

'7113'

In [136]:
node

'7113'

In [133]:
#Permanent cell 12
#Export jpgs

aprx = arcpy.mp.ArcGISProject("CURRENT")
project_path = aprx.filePath

jpg_folder = output_folder + r'\HTML\Maps_And_CSS'
if not os.path.isdir(jpg_folder): os.makedirs(jpg_folder) 
    

# project_directory = os.path.dirname(project_path)

layouts = aprx.listLayouts()

for layout in layouts:

    if layout.mapSeries is not None:
        map_series = layout.mapSeries
        # Loop through all pages in the map series
        for page_number in range(1, map_series.pageCount + 1):
            map_series.currentPageNumber = page_number
            output_filename = os.path.join(jpg_folder, f"{map_series.pageRow.Drains_To}.jpg")
            layout.exportToJPEG(output_filename, resolution=300)
            print (f'Printing jpg {page_number} of {map_series.pageCount}')

print("Export complete.")




Printing jpg 220 of 466
Printing jpg 221 of 466
Printing jpg 222 of 466
Printing jpg 223 of 466
Printing jpg 224 of 466
Printing jpg 225 of 466
Printing jpg 226 of 466
Printing jpg 227 of 466
Printing jpg 228 of 466
Printing jpg 229 of 466


KeyboardInterrupt: 

In [139]:
del layout

In [ ]:
#Permanent cell 13
#Create HTMLs


shutil.copy2('style.css', html_folder + '\\style.css')
shutil.copy2('script.js', html_folder + '\\script.js')

for category in categories:
    area_type = category[0]
    area_names = category[1]
    header_start = category[2]
    
    f = open(html_folder + '\\Population_By_' + area_type + '_' + model_area + '.html', "w")
    f.write('<link rel="stylesheet" href="style.css">\n')
    f.write('<script src="script.js"></script>\n')
    f.write('<link rel="stylesheet" href="style.css">\n')
    f.write('<!DOCTYPE html>\n')
    f.write('<html>\n')
    f.write('<head>\n')
    f.write('<meta charset="utf-8">\n')
    f.write('</head>\n')
    f.write('<body>\n\n')

    f.write('<div class="tab">\n')
    for area_name in area_names:
        tab = area_name

    #     color = ps_dict[first_year]
    #     bg_color = color_dict[color][0]
    #     text_color = color_dict[color][1]

        f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
    f.write('</div>\n')
    
    pop_df = pop_dfss[0][2]

    for area_name in area_names:
        
        area_df = pop_df[pop_df[area_type]==area_name]
        area_df = area_df[['Year','Population']].groupby(['Year']).sum()

        f.write('<div id="' + area_name + '" class="tabcontent">\n') 
        f.write('<h1>' + area_name + '</h1>\n')

        f.write('<div class="sidenav">\n')

        f.write('<table style=\'width: 90%;\'>\n')
        f.write('<tr>\n')
        f.write('<th>Year</th>\n')
        f.write('<th>Population</th>\n')
        f.write('</tr>\n')

        for index, row in area_df.iterrows():
            f.write('<tr>\n')
            f.write('<td>'+ str(index) + '</td>\n')
            population_with_separator = f"{int(row['Population']):,}"
            f.write('<td>'+ population_with_separator + '</td>\n')

            f.write('</tr>\n')
        f.write('</table>\n')
        
        for i in range(4):
            f.write('<h1 style="color: white">End of tables</h1>\n')#Invisible, just to enable scroll to table bottoms

        f.write('</div>\n') #end sidenav


        f.write('<div class="main">\n')

        fig = go.Figure()


        fig.add_trace(go.Scatter(x=area_df.index, 
                                     y = area_df.Population, 
                                     mode='lines',name=pop_dfss[0][0],line=dict(width=5)))
        
        for pop_dfs in pop_dfss[1:]:
            pop_df_past = pop_dfs[2]
            area_df = pop_df_past[pop_df_past[area_type]==area_name]
            area_df = area_df[['Year','Population']].groupby(['Year']).sum()
            fig.add_trace(go.Scatter(x=area_df.index, 
                                     y = area_df.Population, 
                                     mode='lines',name=pop_dfs[0],line=dict(width=2)))
                    
        fig.update_layout(
            title=header_start + area_name,
            autosize=False,
            width = 1500,
            height=850,
            margin=dict(
                l=50,
                r=50,
                b=50,
                t=50,
                pad=4
                ),
                yaxis_title = 'Population'
            )
        
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

        f.write('</div>\n') #end div main  

        f.write('</div>\n')  #end div tab   

        f.write('</body>\n')
    f.write('</html>\n')
    f.close()



In [134]:
jpg_folder

'J:\\SEWER_AREA_MODELS\\FSA\\04_ANALYSIS_WORK\\Model_Result_To_GIS\\Automation\\Rawn_Tool\\Output\\HTML\\Maps_And_CSS'

In [ ]:
# import pandas as pd

# # Sample DataFrame
# data = {
#     'Catchment': [1, 2, 3],
#     'Node': [4, 5, 6]

# }
# accumulation_df = pd.DataFrame(data)

# # Create a MultiIndex with the existing columns
# existing_columns_multiindex = pd.MultiIndex.from_tuples([
#     ('GENERAL INFO', 'Catchment'),  # Header with no subheaders
#     ('GENERAL INFO', 'Node'),  # Header with no subheaders
# ])

# # Create a MultiIndex with the upper level 'GENERAL INFO'
# upper_level = [('GENERAL INFO', '')] * len(existing_columns_multiindex)

# # Concatenate the upper level and the existing columns MultiIndex
# new_columns_multiindex = pd.MultiIndex.from_tuples(list(zip(upper_level, existing_columns_multiindex)))

# # Assign the new MultiIndex to the DataFrame columns
# accumulation_df.columns = new_columns_multiindex

# accumulation_df


In [ ]:
# import pandas as pd

# # Sample data for the DataFrame
# data = {
#     ('GENERAL INFO', 'CATCHMENT'): accumulation_df.Catchment,
#     ('GENERAL INFO', 'NODE'): accumulation_df.Node,
# }

# # Create a DataFrame with MultiIndex columns
# df = pd.DataFrame(data)

# # Set names for the levels of the MultiIndex
# # df.columns.names = ['Header', 'Subheader']

# df